**Homework 21**

In this assignment you will create a Convolutional Neural Network to do facial recognition. We begin by importing the dataset of face photos from the previous assignment.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


from sklearn.datasets import fetch_lfw_people
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
faces=lfw_people.images
names=lfw_people.target_names
target=lfw_people.target

We start with an 80/20 train/test split:

In [2]:
faces_train, faces_test, target_train, target_test=train_test_split(faces,target,train_size=0.8)
faces_train.shape

(1030, 50, 37)


Here are the imports that you will need:

In [3]:
import torch
import torch.nn as nn
from torch.optim import Adam

The faces dataset is already scaled appropriately, so we can skip that step. Next, we convert the training and testing targets to pytorch tensors and modify their shape.

In [4]:
faces_train=torch.tensor(faces_train).float().reshape(-1,1,50,37)
faces_test=torch.tensor(faces_test).float().reshape(-1,1,50,37)
target_train=torch.tensor(target_train)
target_test=torch.tensor(target_test)

Create a CNN!

In [9]:
model=nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Flatten(),
    nn.Linear(128 * 6 * 4, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, len(names))
)

Define an Adam optimizer for this model with an initial learning rate of 0.001.

In [10]:
optimizer=Adam(model.parameters(), lr=0.001)

Defie a training loop to fit your model faces_train and y_train. Train for 20 epochs in batches of size 32. Every other epoch report the accuracy of the model on the test set.

In [11]:
loss_fn = nn.CrossEntropyLoss()
n_epochs=20
N = faces_train.shape[0]  # total number of observations in training data
batch_size=32

for epoch in range(n_epochs):
  model.train()
  # Shuffle the indices
  indices = torch.randperm(N)

  # Create mini-batches
  for i in range(0, N, batch_size):
    batch_indices = indices[i:i+batch_size]
    batch_X = faces_train[batch_indices]
    batch_y = target_train[batch_indices]

    optimizer.zero_grad()
    output = model(batch_X)
    loss = loss_fn(output, batch_y)
    loss.backward()
    optimizer.step()

  if epoch%2==0:
    model.eval()
    with torch.no_grad():
      y_pred=torch.argmax(model(faces_test), dim=1)
      accuracy=(y_pred == target_test).float().mean().item()
      print(f"epoch: {epoch}, accuracy: {accuracy}")

epoch: 0, accuracy: 0.43410852551460266
epoch: 2, accuracy: 0.8255813717842102
epoch: 4, accuracy: 0.9031007885932922
epoch: 6, accuracy: 0.9147287011146545
epoch: 8, accuracy: 0.9418604373931885
epoch: 10, accuracy: 0.9496123790740967
epoch: 12, accuracy: 0.9573643207550049
epoch: 14, accuracy: 0.961240291595459
epoch: 16, accuracy: 0.9496123790740967
epoch: 18, accuracy: 0.9534883499145508


Transfer your final accuracy to the file "heomwork21gradescope.py".